This notebook modify from :

Inference : https://github.com/Prim9000/tacotron2/blob/master/ThaiTTS_Inference.ipynb

Train : https://github.com/Prim9000/Thai_TTS/blob/main/Thai_TTS_Training.ipynb

# Inference

In [1]:
%tensorflow_version 1.x
import os
from os.path import exists, join, basename, splitext
git_repo_url = 'https://github.com/NVIDIA/tacotron2.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  !cd {project_name}/waveglow && git checkout 9168aea
  !pip install -q librosa unidecode
  
import sys
sys.path.append(join(project_name, 'waveglow/'))
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

TensorFlow 1.x selected.
Previous HEAD position was 5bc2a53 README.md: reporting correct number after finding bug in inference time code
HEAD is now at 9168aea README.md: layout
     |████████████████████████████████| 235 kB 9.0 MB/s 


In [2]:
# if download from above don't work run this
!pip install --upgrade gdown

!gdown 1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA
!gdown 1WsibBTsuRg_SF2Z6L6NFRTT-NjEy1oTx
tacotron2_pretrained_model = 'tacotron2_statedict.pt'
waveglow_pretrained_model = '/content/waveglow_256channels_ljs_v2.pt'

Downloading...
From: https://drive.google.com/uc?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA
To: /content/tacotron2_statedict.pt
100% 113M/113M [00:01<00:00, 84.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WsibBTsuRg_SF2Z6L6NFRTT-NjEy1oTx
To: /content/waveglow_256channels_ljs_v2.pt
100% 676M/676M [00:05<00:00, 133MB/s]


In [3]:
import IPython.display as ipd
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT
from audio_processing import griffin_lim
from text import text_to_sequence
from denoiser import Denoiser

def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none', cmap='viridis')

torch.set_grad_enabled(False)
        
# initialize Tacotron2 with the pretrained model
hparams = create_hparams()
hparams.sampling_rate = 22050
model = Tacotron2(hparams)
model.load_state_dict(torch.load(tacotron2_pretrained_model)['state_dict'])
_ = model.cuda().eval()#.half()

# initialize Waveglow with the pretrained model
# waveglow = torch.load(waveglow_pretrained_model)['model']
# WORKAROUND for: https://github.com/NVIDIA/tacotron2/issues/182
import json
from glow import WaveGlow
waveglow_config = json.load(open('%s/waveglow/config.json' % project_name))['waveglow_config']
waveglow = WaveGlow(**waveglow_config)
waveglow.load_state_dict(torch.load(waveglow_pretrained_model)['model'].state_dict())
_ = waveglow.cuda().eval()#.half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



tacotron2/waveglow/glow.py:74: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1937.)
  W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has c

In [4]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

In [5]:
%cd /content/tacotron2

/content/tacotron2


In [6]:
!pip uninstall --yes protobuf

Found existing installation: protobuf 3.17.3
Uninstalling protobuf-3.17.3:
  Successfully uninstalled protobuf-3.17.3


In [7]:
!pip install --no-binary=protobuf protobuf

     |████████████████████████████████| 216 kB 8.1 MB/s 
Skipping wheel build for protobuf, due to binaries being disabled for it.
    Running setup.py install for protobuf ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.5.3 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.


In [8]:
from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser

In [9]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none')

In [10]:
hparams = create_hparams()
hparams.sampling_rate = 22050

In [11]:
# Download weught Checkpoint 8000
!gdown 1-l-GUrLLTGSOgH8zUhzzfaMGz65bn85n

Downloading...
From: https://drive.google.com/uc?id=1-l-GUrLLTGSOgH8zUhzzfaMGz65bn85n
To: /content/tacotron2/checkpoint_8000
100% 338M/338M [00:03<00:00, 104MB/s]


In [12]:
checkpoint_path = "/content/tacotron2/checkpoint_8000"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

In [13]:
waveglow_path = '/content/waveglow_256channels_ljs_v2.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or

# Synthesizing the text 

In [14]:
!pip install pythainlp

     |████████████████████████████████| 11.5 MB 8.7 MB/s 


In [57]:
# text = 'ยินดีที่ได้รู้จัก นี่คือเสียงจากปัญญาประดิษฐ์'

# Input text should be like..
# text = 'th-a-ng^-3 j-a-ng^-0 ch-uua-n^-0 p-a-j^-0 n-a-ng^-2 d-vv-m^-1 l-a-w^-2 th-xx-w^-4 p-a-m^-3 kh-aa-l^-0 th-e-k^-1'
# text = 's-a-z^-0 m-a-j^-4 p-e-n^-0 z-ee-k^-1 z-a-k^-1 khr-a-z^-0 r-aa-t^-2 ch-a-z^-0 th-uu-t^-2 pr-a-z^-1 c-a-m^-0 s-a-z^-0 h-a-z^-1 ph-a-n^-0 s-aa-z^-4 th-aa-z^-0 r-a-z^-0 n-a-z^-3 r-a-t^-3 j-qq-z^-0 r-a-z^-0 m-a-z^-0 n-ii-z^-0'
# text = 'n-x-z^-3 s-aa-ng^-2 r-oo-ng^-0 m-@@-z^-4 kl-a-j^-2 b-aa-n^-2'
# text = 'l-x-z^-3 phr-a-z^-3 th-a-z^-3 w-i-t^-3 m-aa-z^-0 ph-o-p^-3 kh-a-w^-2 c-v-ng^-0 b-a-n^-0 d-aa-n^-0 th-oo-z^-0 s-a-z^-1 khw-aa-z^-3 m-ii-t^-2 z-ii-z^-0 t-oo-z^-2 f-a-n^-0 kh-@@-z^-0 n-aa-j^-0 r-oo-t^-2 w-i-t^-1 kh-aa-t^-1 d-a-p^-1 s-a-z^-1 j-@@-ng^-4 k-@@-n^-1 c-a-z^-1 l-o-p^-1 n-ii-z^-4 p-a-j^-0'
text = 'ch-a-n^-4 r-a-k^-3 th-qq-z^-0 ch-a-n^-4  r-a-k^-3 th-qq-z^-0 ch-a-n^-4  r-a-k^-3 th-qq-z^-0 ch-a-n^-4  r-a-k^-3 th-qq-z^-0'

In [58]:
sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

In [59]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

In [60]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [52]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate)